In [1]:
!pip3 install fasttext scikit-learn numpy pandas seaborn

In [2]:
import numpy as np, fasttext as ft, pandas as pd, seaborn as sns, sklearn as sk

In [3]:
!wget https://dl.fbaipublicfiles.com/fasttext/data/cooking.stackexchange.tar.gz && tar xvzf cooking.stackexchange.tar.gz

--2023-10-14 14:07:02--  https://dl.fbaipublicfiles.com/fasttext/data/cooking.stackexchange.tar.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.244.102.12, 18.244.102.73, 18.244.102.110, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.244.102.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 457609 (447K) [application/x-tar]
Saving to: ‘cooking.stackexchange.tar.gz.2’

cooking.stackexchan 100%[===================>] 446,88K  --.-KB/s    in 0,05s   

2023-10-14 14:07:02 (8,50 MB/s) - ‘cooking.stackexchange.tar.gz.2’ saved [457609/457609]

cooking.stackexchange.id
cooking.stackexchange.txt
readme.txt


In [4]:
!head -n 5 cooking.stackexchange.txt

__label__sauce __label__cheese How much does potato starch affect a cheese sauce recipe?
__label__food-safety __label__acidity Dangerous pathogens capable of growing in acidic environments
__label__cast-iron __label__stove How do I cover up the white spots on my cast iron stove?
__label__restaurant Michelin Three Star Restaurant; but if the chef is not there
__label__knife-skills __label__dicing Without knife skills, how can I quickly and accurately dice vegetables?


In [5]:
!wc -l cooking.stackexchange.txt

15404 cooking.stackexchange.txt


In [6]:
!cat cooking.stackexchange.txt | shuf > cooking.stackexchange.shuffled.txt
!head -n 5 cooking.stackexchange.shuffled.txt

__label__food-safety What precautions should I take when preparing food while I have a cold?
__label__sauce __label__roux __label__ratio What is the thickening power of different types of roux?
__label__chocolate __label__additives Turning raw/baking cocoa powder into instant cocoa (Kaba/Nesquik)
__label__onions __label__hamburgers Cooling cooked onions before adding to raw mince
__label__flavor __label__smell Are there particular palate cleansers for particular foods?


In [7]:
!head -n 12404 cooking.stackexchange.shuffled.txt > cooking.train
!head -n 5 cooking.train

__label__food-safety What precautions should I take when preparing food while I have a cold?
__label__sauce __label__roux __label__ratio What is the thickening power of different types of roux?
__label__chocolate __label__additives Turning raw/baking cocoa powder into instant cocoa (Kaba/Nesquik)
__label__onions __label__hamburgers Cooling cooked onions before adding to raw mince
__label__flavor __label__smell Are there particular palate cleansers for particular foods?


In [8]:
!tail -n 3000 cooking.stackexchange.shuffled.txt > cooking.valid
!head -n 5 cooking.valid

__label__gluten-free __label__french-cuisine __label__cornstarch Hiding the taste of cornstarch in gluten-free bechamel
__label__chicken __label__brining __label__chemistry How deeply will the flavors in a brine penetrate chicken?
__label__coffee __label__roasting Can I use chestnut roaster clay pot to roast coffee at home?
__label__baking __label__brownies Uneven brownie baking?
__label__ravioli Ravioli filling to edge ratio


In [9]:
ft.train_supervised?

Signature: ft.train_supervised(*kargs, **kwargs)
Docstring:
Train a supervised model and return a model object.

input must be a filepath. The input text does not need to be tokenized
as per the tokenize function, but it must be preprocessed and encoded
as UTF-8. You might want to consult standard preprocessing scripts such
as tokenizer.perl mentioned here: http://www.statmt.org/wmt07/baseline.html

The input file must must contain at least one label per line. For an
example consult the example datasets which are part of the fastText
repository such as the dataset pulled by classification-example.sh.
File:      ~/.local/lib/python3.8/site-packages/fasttext/FastText.py
Type:      function

In [10]:
lr_step = 3
lr_values = [v / 100 for v in range(1, 100 + lr_step, lr_step)]
lr_values[:5]

[0.01, 0.04, 0.07, 0.1, 0.13]

In [11]:
lr_values[-5:]

[0.88, 0.91, 0.94, 0.97, 1.0]

In [12]:
e_step = 1
epoch_values = list(range(5, 50+e_step, e_step))
epoch_values[:5]

[5, 6, 7, 8, 9]

In [13]:
epoch_values[-5:]

[46, 47, 48, 49, 50]

In [1]:
ngrams_values = list(range(1, 6))
ngrams_values

[1, 2, 3, 4, 5]

In [2]:
# HYPER PARAMS
# lr=1.0, # [0.1 - 1.0]
# epoch=25, # [5-50]
# wordNgrams=2 #[1-5]

In [24]:
model = ft.train_supervised(input="cooking.train", autotuneValidationFile='cooking.valid', autotuneDuration=600, thread=16)

Progress: 100.0% Trials:   23 Best score:  0.375484 ETA:   0h 0m 0s
Training again with best arguments
Read 0M words
Number of words:  14562
Number of labels: 735
Progress: 100.0% words/sec/thread:    8996 lr:  0.000000 avg.loss:  3.754475 ETA:   0h 0m 0s


In [16]:
def test_model(model):
    count, prec, rec = model.test("cooking.valid")
    f1 = 2 * prec * rec / (prec + rec)
    print(f"Count: {count}, f1: {f1} prec: {prec}, rec: {rec}, ")

In [17]:
test_model(model)

Count: 2999, f1: 0.35060167244544155 prec: 0.573191063687896, rec: 0.25253415601586604, 


In [18]:
fat_model_path = 'cooking_model.bin'
model.save_model(fat_model_path)

In [19]:
!ls -lah {fat_model_path}

-rw-rw-r-- 1 sun sun 770M жов 14 14:07 cooking_model.bin


In [20]:
model.quantize?

Signature:
model.quantize(
    input=None,
    qout=False,
    cutoff=0,
    retrain=False,
    epoch=None,
    lr=None,
    thread=None,
    verbose=None,
    dsub=2,
    qnorm=False,
)
Docstring:
Quantize the model reducing the size of the model and
it's memory footprint.
File:      ~/.local/lib/python3.8/site-packages/fasttext/FastText.py
Type:      method

In [21]:
model.quantize(input="cooking.train", qnorm=True, retrain=True, cutoff=100000)

Progress: 100.0% words/sec/thread:   23056 lr:  0.000000 avg.loss:  1.942648 ETA:   0h 0m 0s


In [22]:
test_model(model)

Count: 2999, f1: 0.35957576993677337 prec: 0.5878626208736245, rec: 0.2589980902012634, 


In [23]:
thin_model_path = 'cooking_model.ftz'
model.save_model(thin_model_path)
!ls -lah {thin_model_path}

-rw-rw-r-- 1 sun sun 6,2M жов 14 14:08 cooking_model.ftz
